In [ ]:
import copy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle

In [ ]:
train_df = pd.read_parquet('../data/processed/train.parquet')
dev_df = pd.read_parquet('../data/processed/dev.parquet')
test_df = pd.read_parquet('../data/processed/test.parquet')

print(f"Loaded Train: {train_df.shape}, Dev: {dev_df.shape}")

In [ ]:
MODEL_NAME = "UBC-NLP/MARBERTv2"
MAX_LEN = 64
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


le_multi = LabelEncoder()


le_multi.fit(train_df["stratify_label"])


print(f"Classes found: {le_multi.classes_}")


class HateSpeechMTLDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, multi_label_encoder):
        self.texts = df["text_clean"].astype(str).to_numpy()
        self.binary_labels = df["is_hate"].to_numpy()
        self.multi_labels = multi_label_encoder.transform(df["stratify_label"])
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "binary_label": torch.tensor(self.binary_labels[item], dtype=torch.float),
            "multi_label": torch.tensor(self.multi_labels[item], dtype=torch.long),
        }


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = HateSpeechMTLDataset(train_df, tokenizer, MAX_LEN, le_multi)

val_dataset = HateSpeechMTLDataset(dev_df, tokenizer, MAX_LEN, le_multi)


# test_dataset = HateSpeechMTLDataset(test_df, tokenizer, MAX_LEN, le_multi)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
print("Data Loaders Ready.")

In [ ]:
class MTL_Marbert(nn.Module):
    def __init__(self, model_name, n_multi_classes):
        super(MTL_Marbert, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.3)
        self.binary_head = nn.Linear(self.bert.config.hidden_size, 1)
        self.multi_head = nn.Linear(self.bert.config.hidden_size, n_multi_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        pooled_output = outputs.last_hidden_state[:, 0, :]
        output = self.drop(pooled_output)
        binary_logits = self.binary_head(output)
        multi_logits = self.multi_head(output)

        return binary_logits, multi_logits


model = MTL_Marbert(MODEL_NAME, n_multi_classes=len(le_multi.classes_))
model = model.to(DEVICE)
print("MTL Model Initialized.")

In [ ]:
criterion_binary = nn.BCEWithLogitsLoss()
criterion_multi = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    losses = []

    for d in tqdm(data_loader, leave=False):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)

        binary_targets = d["binary_label"].to(device)
        multi_targets = d["multi_label"].to(device)

        binary_logits, multi_logits = model(input_ids, attention_mask)

        loss_b = criterion_binary(binary_logits, binary_targets.unsqueeze(1))
        loss_m = criterion_multi(multi_logits, multi_targets)

        loss = loss_b + loss_m

        optimizer.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())

    return np.mean(losses)

print("Setup Complete")


In [ ]:
PATIENCE = 3
best_val_loss = float("inf")
counter = 0
best_model_weights = copy.deepcopy(model.state_dict())


def eval_model(model, data_loader, device):
    model = model.eval()
    fin_binary_preds, fin_multi_preds = [], []
    fin_binary_targets, fin_multi_targets = [], []
    total_loss = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            binary_targets = d["binary_label"].to(device)
            multi_targets = d["multi_label"].to(device)
            binary_logits, multi_logits = model(input_ids, attention_mask)

            loss_b = criterion_binary(binary_logits, binary_targets.unsqueeze(1))
            loss_m = criterion_multi(multi_logits, multi_targets)
            loss = loss_b + loss_m
            total_loss += loss.item()
            binary_preds = torch.sigmoid(binary_logits).cpu().detach().numpy() > 0.5
            multi_preds = (
                torch.softmax(multi_logits, dim=1).argmax(dim=1).cpu().detach().numpy()
            )
            fin_binary_preds.extend(binary_preds.flatten())
            fin_multi_preds.extend(multi_preds.flatten())
            fin_binary_targets.extend(binary_targets.cpu().detach().numpy())
            fin_multi_targets.extend(multi_targets.cpu().detach().numpy())

    avg_loss = total_loss / len(data_loader)
    bin_f1 = f1_score(fin_binary_targets, fin_binary_preds, average="macro")
    multi_f1 = f1_score(fin_multi_targets, fin_multi_preds, average="macro")

    return bin_f1, multi_f1, avg_loss


print(f"Starting Training with Early Stopping (Patience={PATIENCE})...")

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    train_loss = train_epoch(model, train_loader, optimizer, DEVICE, scheduler)

    val_bin_f1, val_multi_f1, val_loss = eval_model(model, val_loader, DEVICE)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Val Binary F1: {val_bin_f1:.4f} | Val Multi F1: {val_multi_f1:.4f}")

    if val_loss < best_val_loss:
        print(
            f"✅ Validation Loss Improved ({best_val_loss:.4f} -> {val_loss:.4f}). Saving model..."
        )
        best_val_loss = val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        counter = 0
        torch.save(model.state_dict(), "best_mtl_marbert.pth")

    else:
        counter += 1
        print(f"⚠️ No improvement. EarlyStopping counter: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("🛑 Early Stopping triggered! Restoring best weights...")
            model.load_state_dict(best_model_weights)
            break


print("Training Complete.")


In [ ]:
MTL_DIR = '../models/multitask_marbert'
os.makedirs(MTL_DIR, exist_ok=True)

print(f"Saving Multi-Task System to: {MTL_DIR} ...")

# Save Model Weights 
weights_path = os.path.join(MTL_DIR, 'model_weights.pth')
torch.save(model.state_dict(), weights_path)
print(f"Model weights saved to: {weights_path}")

# Save Tokenizer 
tokenizer.save_pretrained(MTL_DIR)
print(f"Tokenizer saved to: {MTL_DIR}")

# Save Label Encoder 
le_path = os.path.join(MTL_DIR, 'label_encoder.pkl')
with open(le_path, 'wb') as f:
    pickle.dump(le_multi, f)
print(f"Label Encoder saved to: {le_path}")

# Save Config 
config_path = os.path.join(MTL_DIR, 'config.pkl')
config_data = {
    'base_model': MODEL_NAME,
    'max_len': MAX_LEN,
    'num_classes': len(le_multi.classes_)
}
with open(config_path, 'wb') as f:
    pickle.dump(config_data, f)
print(f"Config saved to: {config_path}")

print("\n Multi-Task System export complete.")

In [ ]:
def generate_detailed_report(
    model, data_loader, device, label_encoder, dataset_name="Validation"
):
    model.eval()
    all_bin_preds = []
    all_bin_targets = []
    all_multi_preds = []
    all_multi_targets = []

    print(f"Generating Report for {dataset_name} Set...")

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            binary_targets = d["binary_label"].to(device)
            multi_targets = d["multi_label"].to(device)
            binary_logits, multi_logits = model(input_ids, attention_mask)
            binary_preds = (
                (torch.sigmoid(binary_logits) > 0.5).int().cpu().numpy().flatten()
            )
            multi_preds = (
                torch.softmax(multi_logits, dim=1).argmax(dim=1).cpu().numpy().flatten()
            )
            all_bin_preds.extend(binary_preds)
            all_bin_targets.extend(binary_targets.cpu().numpy().astype(int))
            all_multi_preds.extend(multi_preds)
            all_multi_targets.extend(multi_targets.cpu().numpy())

    print("\n" + "=" * 60)
    print("📊 TASK A: BINARY HATE DETECTION (Hate vs Not Hate)")
    print("=" * 60)
    print(
        classification_report(
            all_bin_targets, all_bin_preds, target_names=["Not Hate", "Hate"], digits=4
        )
    )

    print(
        f"Overall Binary Accuracy: {accuracy_score(all_bin_targets, all_bin_preds):.4f}"
    )
    print("\n" + "=" * 60)
    print(
        f"📊 TASK B: FINE-GRAINED CLASSIFICATION ({', '.join(label_encoder.classes_)})"
    )
    print("=" * 60)
    print(
        classification_report(
            all_multi_targets,
            all_multi_preds,
            target_names=label_encoder.classes_,
            digits=4,
        )
    )

    print(
        f"Overall Multi-Class Accuracy: {accuracy_score(all_multi_targets, all_multi_preds):.4f}"
    )

    plt.figure(figsize=(8, 6))

    cm = confusion_matrix(all_multi_targets, all_multi_preds)

    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=label_encoder.classes_,
        yticklabels=label_encoder.classes_,
    )

    plt.title("Confusion Matrix - Fine Grained Task")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()


generate_detailed_report(model, val_loader, DEVICE, le_multi, "Validation")


test_dataset = HateSpeechMTLDataset(test_df, tokenizer, MAX_LEN, le_multi)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
generate_detailed_report(model, test_loader, DEVICE, le_multi, "Test")
